In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import time

In [3]:
from IPython.display import HTML

In [4]:
from brightness_controller import BrightnessController
from controller import Controller
from mpv import MPVController
from radar import LD2450
from radar_data_collector import RadarDataCollector
from sliding_average import SlidingAverage

In [5]:
video_fpath = "/home/tami/videos/head_on_floor_1920x1080.mp4"

uartdev = "/dev/ttyAMA0"

ws = 1
dmin = 1000
dmax = 2500

i2c_num = 2

## Sliding average

In [6]:
SlidingAverage.show_check(n=12, ws=4)

 -32 |  -32    0    0    0 |  -32    0    0    0 |  -8.00  -8.00 | yes
 -42 |  -32  -42    0    0 |  -32  -42    0    0 | -18.50 -18.50 | yes
  83 |  -32  -42   83    0 |  -32  -42   83    0 |   2.25   2.25 | yes
   8 |  -32  -42   83    8 |  -32  -42   83    8 |   4.25   4.25 | yes
 -33 |  -42   83    8  -33 |  -33  -42   83    8 |   4.00   4.00 | yes
  59 |   83    8  -33   59 |  -33   59   83    8 |  29.25  29.25 | yes
 -97 |    8  -33   59  -97 |  -33   59  -97    8 | -15.75 -15.75 | yes
  52 |  -33   59  -97   52 |  -33   59  -97   52 |  -4.75  -4.75 | yes
  68 |   59  -97   52   68 |   68   59  -97   52 |  20.50  20.50 | yes
  -4 |  -97   52   68   -4 |   68   -4  -97   52 |   4.75   4.75 | yes
  66 |   52   68   -4   66 |   68   -4   66   52 |  45.50  45.50 | yes
 -50 |   68   -4   66  -50 |   68   -4   66  -50 |  20.00  20.00 | yes


## Radar LD2450

In [11]:
r = LD2450(uartdev)

In [ ]:
r.set_bluetooth_on(restart=True)

In [ ]:
r.set_multi_tracking()

In [ ]:
r.show_info()

In [12]:
r.test()

In [ ]:
r.show_data(20)

## Radar 2410C

In [13]:
# ...

## Radar 2461

## Radar data collector

In [21]:
rdc = RadarDataCollector(uartdev)

TypeError: RadarDataCollector.__init__() missing 1 required positional argument: 'uartdev'

## Camera

In [15]:
# ...

## Direct brightness

In [5]:
br_ctl = BrightnessController(i2c_num)

In [9]:
for i in range(0, 101):
    br_ctl.set(i)
    time.sleep(0.01)

## Manipulating gamma

In [1]:
# ...

## MPV

In [6]:
mpv_ctl = MPVController()

In [7]:
# mpv_ctl.get_property("working-directory")

In [8]:
# mpv_ctl.get_property("term-status-msg")

In [9]:
# mpv_ctl.socket.recv(1024)

In [10]:
# problems = []
# for i in range(500):
#     res = mpv_ctl.load_file(video_fpath)
#     if res is not None and res['error'] == 'success':
#         continue

#     problems.append(i)

# print(problems)

In [142]:
mpv_ctl.set_property("loop-file", "inf")

{'request_id': 902, 'error': 'success'}

In [21]:
mpv_ctl.clear()

{'data': None, 'request_id': 1, 'error': 'success'}

In [11]:
mpv_ctl.load_file(video_fpath)

{'data': {'playlist_entry_id': 1}, 'request_id': 1, 'error': 'success'}

In [14]:
mpv_ctl.get_property("duration")

{'data': 888.48, 'request_id': 4, 'error': 'success'}

In [15]:
mpv_ctl.get_property("playlist/count")

{'data': 1, 'request_id': 5, 'error': 'success'}

In [16]:
mpv_ctl.get_property("playlist/0/filename")

{'data': '/home/tami/videos/head_on_floor_1920x1080.mp4',
 'request_id': 6,
 'error': 'success'}

In [17]:
mpv_ctl.play()

{'request_id': 7, 'error': 'success'}

In [18]:
mpv_ctl.get_property("pause")

{'data': False, 'request_id': 8, 'error': 'success'}

In [12]:
mpv_ctl.pause()

{'request_id': 2, 'error': 'success'}

In [140]:
mpv_ctl.get_property("pause")

{'data': False, 'request_id': 900, 'error': 'success'}

In [139]:
mpv_ctl.get_property("time-pos")

{'data': 4.541583, 'request_id': 899, 'error': 'success'}

In [75]:
mpv_ctl.get_property("playback-time")

{'data': -0.004625, 'request_id': 634, 'error': 'success'}

In [138]:
mpv_ctl.get_property("time-remaining")

{'data': 9.893542, 'request_id': 898, 'error': 'success'}

In [137]:
mpv_ctl.get_property("framedrop")

{'data': 'vo', 'request_id': 897, 'error': 'success'}

In [136]:
mpv_ctl.get_property("frame-drop-count")

{'data': 4, 'request_id': 896, 'error': 'success'}

In [113]:
mpv_ctl.seek(0)

{'data': None, 'request_id': 873, 'error': 'success'}

In [116]:
mpv_ctl.get_property("playback-time")

{'data': 7.87825, 'request_id': 876, 'error': 'success'}

In [110]:
mpv_ctl.get_property("time-remaining")

{'data': 4.012667, 'request_id': 870, 'error': 'success'}

In [145]:
for i in range(-100, 1):
    mpv_ctl.set_brightness(i, osd=True)
    time.sleep(0.1)

In [153]:
mpv_ctl.get_property("volume")

{'data': 100.0, 'request_id': 115, 'error': 'success'}

## Controller

In [ ]:
ctl = Controller(uartdev, ws, dmin, dmax, screen_bus=i2c_num)

In [ ]:
ctl.start()

## Plotter

In [ ]:
p = Plotter(uartdev, ws, dmin, dmax, verbose=True, xmin=-3000, xmax=3000, ymin=-4000)

In [ ]:
ani = p.start()

In [ ]:
HTML(ani.to_jshtml())